In [95]:
import pandas as pd
import plotly.offline as py
import plotly.graph_objs as go

py.offline.init_notebook_mode(connected=True)

In [96]:
locations = pd.read_csv('locations.csv')
pickup_times =pd.read_csv('pickup_times.csv')

## Calculating location medians

In [148]:
def locations_median(day, start_hour, end_hour):
    locations = pd.read_csv('locations.csv')
    pickup_times =pd.read_csv('pickup_times.csv')

    start = '' + day + 'T' + start_hour + 'Z'
    end = '' + day + 'T' + end_hour + 'Z'

    pickup_times_slice = pickup_times[(pickup_times['iso_8601_timestamp'] > start) 
    & (pickup_times['iso_8601_timestamp'] <= end)]

    pickup_times_by_location = pickup_times_slice.groupby('location_id')
    location_medians = pickup_times_by_location['pickup_time'].median()

    location_indexes = location_medians.index.values
    location_medians = location_medians

    location_medians_with_index = pd.DataFrame(data={'location_id': location_indexes, 'median': location_medians},
     columns=['location_id', 'median'])
    
    joined = location_medians_with_index.merge(locations, left_on='location_id', right_on='location_id', how='outer')

    return joined

In [150]:
example_location_medians = locations_median('2019-01-13', '15:00', '16:00')

In [151]:
example_location_medians.head()

location_id  median  longitude   latitude
0            1    22.0  24.941583  60.170870
1            2    29.0  24.934201  60.169802
2            3    16.5  24.938493  60.166408
3            4    29.0  24.940467  60.167048
4            5    20.0  24.938793  60.167518

## Visualizing location medians

In [146]:
def formation_location_text(location_data):
    location_text = []
    
    locations = location_data.location_id
    medians = location_data['median']

    for row in location_data.iterrows(): 
        text = 'id: ' + str(row[1]['location_id']) + ' Md: ' + str(row[1]['median']) + ''
        location_text.append(text)
            
    return location_text

def visualize_location_medians(location_data):
    
    data = [
    go.Scatter(
        x=location_data.longitude,
        y=location_data.latitude,
        mode = 'markers',
        text = formation_location_text(location_data),
        marker=dict(
            size=10,
            color=location_data['median'],
            colorbar=dict(
                title='Colorbar',
                dtick =2),
            colorscale='RdBu'
            ),
        ),
    ]
    
    layout = go.Layout(
        title= 'Helsinki restaurant median pickup times',
        xaxis=dict(
        title='Longitude',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        ),
        showticklabels=True
        
    ),
    yaxis=dict(
        title='Latitude',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        ),
        showticklabels=True
        )
    )
    
    fig = go.Figure(data=data, layout=layout)
    py.iplot(fig)
    


In [147]:
visualize_location_medians(example_location_medians)

## Location medians to csv

In [104]:
def export_to_csv(locations, filename):
    locations.to_csv(filename, index=False)    

In [105]:
export_to_csv(example_location_medians, 'example-location-medians.csv')